In [ ]:
#Packages/libary requests, pandas, googlemaps
#ip address: XXXX
# Replace YOUR_API_KEY with your actual API key
#removing radius will pull all the restaruents

In [64]:
#Connect to Azure DB
#Think of this as "From this librarby import these methods"
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os


In [70]:
#Extract the Places API data, Create Table, Convert to CSV to move to Azure
import requests
import pandas as pd

latitude = 29.7604
longitude = -95.3698
api_key = "XXXXX"

def fetch_places_data(url):
    response = requests.get(url)
    data = response.json()
    places = data.get("results", [])

    results = []
    for place in places:
        place_id = place.get("place_id")
        
        # Get place details using Place Details API
        details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&
        fields=name,business_status,formatted_address,formatted_phone_number,price_level,rating,reviews,user_ratings_total,website,opening_hours&key={api_key}"
        details_response = requests.get(details_url)
        details_data = details_response.json()
        
        result = {
            "name": details_data["result"].get("name"),
            "business_status": details_data["result"].get("business_status"),
            "formatted_address": details_data["result"].get("formatted_address"),
            "formatted_phone_number": details_data["result"].get("formatted_phone_number"),
            "price_level": details_data["result"].get("price_level"),
            "rating": details_data["result"].get("rating"),
            #"reviews": details_data["result"].get("reviews"),#
            "user_ratings_total": details_data["result"].get("user_ratings_total"),
            "website": details_data["result"].get("website"),
            #"opening_hours": details_data["result"].get("opening_hours"),#
        }
        results.append(result)

    next_page_token = data.get("next_page_token")
    return results, next_page_token

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius=20000&type=restaurant&key={api_key}"

# Fetch the first page of results
results, next_page_token = fetch_places_data(url)

# Fetch additional pages until all results are retrieved
while next_page_token:
    next_url = f"{url}&pagetoken={next_page_token}"
    additional_results, next_page_token = fetch_places_data(next_url)
    results.extend(additional_results)

# Create a DataFrame from the list of results
df = pd.DataFrame(results)


In [66]:
#Count number of records
print("Number of records:", len(df))

Number of records: 60


In [67]:
#Created our csv to move to our Azure Storage account
csv_data = df.to_csv(index=False)

In [ ]:
# Your Azure Storage account connection string
connection_string = "DefaultEndpointsProtocol=https;
AccountName=storageaccount;AccountKey=XXXX==;EndpointSuffix=core.windows.net"

# Your Blob container name
container_name = "storagename"

# Your Blob file name (optional)
blob_name = "blob1.csv"

# Create a BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Create a new BlobClient to upload the data
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the CSV data to the Blob
blob_client.upload_blob(csv_data)
